## Notebook for scrapping data for NHL game prediction.

In [1]:
import requests
import pandas as pd
import numpy as np
import json
import time
!pip install hockey_scraper
import hockey_scraper
from sklearn.preprocessing import LabelEncoder

     |████████████████████████████████| 86 kB 2.7 MB/s 


### scrapping data for 6 seasons starting from 2015 to 2021.

### Scrapping player and goal keeper Id's.

In [122]:
years=['20152016','20162017','20172018','20182019', '20192020','20202021']
athleteid_url='http://{}.web.nhl.com/{}/{}/{}?expand={}.{}'
id_data=json.loads((requests.get(athleteid_url.format('statsapi','api','v1','teams','team','roster'))).content)
athlete_id=pd.DataFrame([])

In [123]:
id_teams=id_data["teams"]

In [124]:
full_name_list=[]
playerid_list=[]
playerposition_list=[]
playerposition_code_list=[]
for i in id_teams:
  for j in i["roster"]["roster"]:
    full_name_list.append(j["person"]["fullName"])
    playerid_list.append(j["person"]["id"])
    playerposition_list.append(j["position"]["name"])
    playerposition_code_list.append(j["position"]["code"])
athlete_id["fullName"]=full_name_list
athlete_id["id"]=playerid_list
athlete_id["position"]=playerposition_list
athlete_id["position_code"]=playerposition_code_list

In [125]:
athlete_id.head()

,fullName,id,position,position_code
0,Jonathan Bernier,8473541,Goalie,G
1,Mackenzie Blackwood,8478406,Goalie,G
2,P.K. Subban,8474056,Defenseman,D
3,Tomas Tatar,8475193,Left Wing,L
4,Dougie Hamilton,8476462,Defenseman,D


In [126]:
goalkeepers_id=athlete_id[athlete_id["position_code"]=="G"]

In [127]:
goalkeepers_id.head()

,fullName,id,position,position_code
0,Jonathan Bernier,8473541,Goalie,G
1,Mackenzie Blackwood,8478406,Goalie,G
5,Jon Gillies,8476903,Goalie,G
8,Andrew Hammond,8477202,Goalie,G
25,Nico Daws,8482076,Goalie,G


In [128]:
athlete_id.to_csv("athlete_id.csv")
goalkeepers_id.to_csv("goalkeepers_id.csv")

### Scarpping Goalkeeper performance data.

In [129]:
goalkeeper_fulldata={}
output_data=pd.DataFrame([])

### Running scrapping loop with a time lapse to avoid IP ban
### Below code cell takes ateast 2 hours to run. Don't run this, Please skip to next cell Where I load the saved files from previous runs. 

In [130]:
for l in range(0,5,2):
  goalkeeper_fulldata={}
  output_data=pd.DataFrame([])
  for i in range(l,l+1):
    for j in range(len(goalkeepers_id)):
      output=pd.read_html('https://www.naturalstattrick.com/{}.php?{}={}&{}={}&{}={}&sit={}&stype={}&stdoi={}&rate={}&v={}'.format('playerreport','fromseason',years[i],'thruseason',years[i+1],'playerid',goalkeepers_id.iloc[j]["id"],'all','2','oi','n','g'))[0]
      if j==0:
        output['id']=goalkeepers_id.iloc[i]["id"]
        output['fullName']=goalkeepers_id.iloc[i]["fullName"]
        output_final=output
      else:
        output['id']=goalkeepers_id.iloc[i]["id"]
        output['fullName']=goalkeepers_id.iloc[i]["fullName"]
        output_final=pd.concat([output,output_final])
    output_data=output_final
    time.sleep(7)
  goalkeeper_fulldata[years[i]+'_'+years[i+1]]=output_data
  time.sleep(15)
goalkeeper_fulldata["20152016_20162017"].to_csv("goalkeeper_fulldata_20152016_20162017.csv")
goalkeeper_fulldata["20172018_20182019"].to_csv("goalkeeper_fulldata_20172018_20182019.csv")
goalkeeper_fulldata["20192020_20202021"].to_csv("goalkeeper_fulldata_20192020_20202021.csv")

In [131]:
data1_dummy=pd.read_csv("goalkeeper_fulldata_20152016_20162017.csv")
data2_dummy=pd.read_csv("goalkeeper_fulldata_20172018_20182019.csv")
data3_dummy=pd.read_csv("goalkeeper_fulldata_20192020_20202021.csv")

In [132]:
goalkeeper_fulldata_df=pd.DataFrame([])
marged_dataframe=[data1_dummy,data2_dummy,data3_dummy]

In [133]:
goalkeeper_fulldata=pd.concat(marged_dataframe,ignore_index=True)

In [134]:
goalkeeper_fulldata["Game"][2] 

'2015-10-23 WSH at EDM'

In [135]:
dummy_list1=[]
for i in range(len(goalkeeper_fulldata["Team"])):
  dummy_list1.append(goalkeeper_fulldata["Team"][i]+' + '+goalkeeper_fulldata["Game"][i][0:10])

In [136]:
goalkeeper_fulldata["Team+day"]=dummy_list1

In [137]:
dummy_list2=[]
for i in range(len(goalkeeper_fulldata["Team"])):
  dummy_list2.append(goalkeeper_fulldata["Game"][i][0:10])

In [138]:
goalkeeper_fulldata["day"]=dummy_list2

In [139]:
goalkeeper_fulldata.head()

,Unnamed: 0,Game,Team,TOI,CF,CA,CF%,FF,FA,FF%,...,On The Fly Starts,Off. Zone Start %,Off. Zone Faceoffs,Neu. Zone Faceoffs,Def. Zone Faceoffs,Off. Zone Faceoff %,id,fullName,Team+day,day
0,0,2015-12-12 OTT at MTL,OTT,31.816667,37,21,63.79,22,14,61.11,...,0,-,13,13,8,61.9,8473541,Jonathan Bernier,OTT + 2015-12-12,2015-12-12
1,1,2016-10-28 OTT at CGY,OTT,40.000000,43,39,52.44,33,26,55.93,...,0,-,23,10,13,63.89,8473541,Jonathan Bernier,OTT + 2016-10-28,2016-10-28
2,0,2015-10-23 WSH at EDM,WSH,59.666667,48,55,46.6,37,39,48.68,...,0,100.00,13,26,23,36.11,8473541,Jonathan Bernier,WSH + 2015-10-23,2015-10-23
3,1,2015-10-31 WSH at FLA,WSH,63.733333,48,44,52.17,36,38,48.65,...,0,-,15,22,18,45.45,8473541,Jonathan Bernier,WSH + 2015-10-31,2015-10-31
4,2,2015-11-13 CGY at WSH,WSH,61.666667,75,55,57.69,48,45,51.61,...,0,-,22,23,20,52.38,8473541,Jonathan Bernier,WSH + 2015-11-13,2015-11-13


### Since Ice Hockey is a low-scoring game. To smoothen out the varience in the data, we are calculating rolling averages of the data. 
###We have calculated 40 an 5 rolling game averages. In the end to include all the opponents a team might encounter, we have decided to use 40 games average. 

In [140]:
## Calculating rolling averages for each column

In [141]:
goalkeeper_fulldata["TOI_MovingAverage"]=goalkeeper_fulldata.groupby('id')['TOI'].apply(lambda x: x.rolling(window=40,min_periods=10).sum().shift())

In [142]:
goalkeeper_fulldata["SA_MovingAverage"]=goalkeeper_fulldata.groupby('id')['SA'].apply(lambda x: x.rolling(window=40,min_periods=10).sum().shift())

In [143]:
goalkeeper_fulldata["FA_MovingAverage"]=goalkeeper_fulldata.groupby('id')['FA'].apply(lambda x: x.rolling(window=40,min_periods=10).sum().shift())

In [144]:
goalkeeper_fulldata["GA_MovingAverage"]=goalkeeper_fulldata.groupby('id')['GA'].apply(lambda x: x.rolling(window=40,min_periods=10).sum().shift())

In [145]:
goalkeeper_fulldata['xGA_MovingAverage']=goalkeeper_fulldata.groupby('id')['xGA'].transform(lambda x: x.rolling(window=40,min_periods=10).sum().shift())

In [146]:
goalkeeper_fulldata["HDCA_MovingAverage"]=goalkeeper_fulldata.groupby('id')['HDCA'].apply(lambda x: x.rolling(window=40,min_periods=10).sum().shift())

In [147]:
goalkeeper_fulldata["HDGA_MovingAverage"]=goalkeeper_fulldata.groupby('id')['HDGA'].apply(lambda x: x.rolling(window=40,min_periods=10).sum().shift())

In [148]:
goalkeeper_fulldata['fwkSV%_GoalKeeper']=(goalkeeper_fulldata['FA_MovingAverage']-goalkeeper_fulldata['GA_MovingAverage'])/goalkeeper_fulldata['FA_MovingAverage']

In [149]:
goalkeeper_fulldata['GSAx_GoalKeeper']=(goalkeeper_fulldata['xGA_MovingAverage']*60/goalkeeper_fulldata['TOI_MovingAverage'])

In [150]:
goalkeeper_fulldata['GSAx/60_GoalKeeper']=goalkeeper_fulldata['GSAx_GoalKeeper']*60/goalkeeper_fulldata['TOI_MovingAverage']
goalkeeper_fulldata['HDCSV%__GoalKeeper']=(goalkeeper_fulldata['HDCA_MovingAverage']-goalkeeper_fulldata['HDGA_MovingAverage'])/goalkeeper_fulldata['HDCA_MovingAverage'] 

In [151]:
goalkeeper_fulldata.to_csv("goalkeeper_fulldata_complete.csv")

In [152]:
goalkeeper_fulldata.head()

,Unnamed: 0,Game,Team,TOI,CF,CA,CF%,FF,FA,FF%,...,SA_MovingAverage,FA_MovingAverage,GA_MovingAverage,xGA_MovingAverage,HDCA_MovingAverage,HDGA_MovingAverage,fwkSV%_GoalKeeper,GSAx_GoalKeeper,GSAx/60_GoalKeeper,HDCSV%__GoalKeeper
0,0,2015-12-12 OTT at MTL,OTT,31.816667,37,21,63.79,22,14,61.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2016-10-28 OTT at CGY,OTT,40.000000,43,39,52.44,33,26,55.93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2015-10-23 WSH at EDM,WSH,59.666667,48,55,46.6,37,39,48.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2015-10-31 WSH at FLA,WSH,63.733333,48,44,52.17,36,38,48.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,2015-11-13 CGY at WSH,WSH,61.666667,75,55,57.69,48,45,51.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
ff_calc=(goalkeeper_fulldata['FA'].sum()-goalkeeper_fulldata['GA'].sum())/goalkeeper_fulldata['FA'].sum()
GSAx_calc=goalkeeper_fulldata['xGA'].sum()-goalkeeper_fulldata['GA'].sum()
GSAx60_calc=(GSAx_calc*60)/goalkeeper_fulldata['TOI'].sum()
HDCSV_calc=(goalkeeper_fulldata['HDCA'].sum()-goalkeeper_fulldata['HDGA'].sum() )/goalkeeper_fulldata['HDCA'].sum()

### Scrapping stats of every game from 2015 to 2021 in all three major formats (5v5, Powerplay, Powerkill).

In [154]:
play=["sva","pp","pk"]

In [155]:
play_stattrick_url='https://www.naturalstattrick.com/games.php?{}={}&{}={}&stype={}&sit={}&loc={}&team={}&rate={}'

In [156]:
teamshortids_dict=open("teamshortids.json")
teamshortids_dict=json.load(teamshortids_dict)

In [157]:
teamid_url='https://statsapi.web.nhl.com/api/v1/teams'
teamid_data=json.loads((requests.get(teamid_url)).content)

In [220]:
teamshortids_dict["Seattle Kraken"]

'SN'

In [229]:
final_data={}
for i in range(len(years)):
  for j in range(len(play)):
    output=pd.read_html(play_stattrick_url.format('fromseason',years[i],'thruseason',years[i],play[j],'2','B','All','n'))    
    for i1 in range(len(output[0]["Team"])):
      output[0]["Team"][i1]=teamshortids_dict[output[0]["Team"][i1]]
    dummy_list11=[]
    for i2 in range(len(output[0]["Team"])):
      dummy_list11.append(output[0]["Team"][i2]+' + '+output[0]['Game'][i2][0:10])
    dummy_list12=[]
    for i3 in range(len(output[0]["Team"])):
      dummy_list12.append(output[0]["Game"][i3][0:10])
    output[0]["day"]=dummy_list12
    output[0]["Team+day"]=dummy_list11
    final_data[years[i]+"_"+play[j]]=output[0]
    time.sleep(3)
  time.sleep(5)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [159]:
finaldata_merged=pd.DataFrame([])
list1_merge=['Team+day', 'TOI', 'xGF', 'GF']
list2_merge=['Team+day', 'TOI', 'xGA', 'GA']

In [160]:
merge_20152016=final_data["20152016_sva"].merge(final_data["20152016_pp"][list1_merge],left_index=False,right_index=False,suffixes=("","_pp"),how='left',on='Team+day',indicator=False,validate=None,)
merge_20152016=merge_20152016.merge(final_data["20152016_pk"][list2_merge],left_index=False,right_index=False,suffixes=("","_pk"),how='left',on='Team+day',indicator=False,validate=None,)

In [161]:
merge_20162017=final_data["20162017_sva"].merge(final_data["20162017_pp"][list1_merge],left_index=False,right_index=False,suffixes=("","_pp"),how='left',on='Team+day',indicator=False,validate=None,)
merge_20162017=merge_20162017.merge(final_data["20162017_pk"][list2_merge],left_index=False,right_index=False,suffixes=("","_pk"),how='left',on='Team+day',indicator=False,validate=None,)

In [162]:
merge_20172018=final_data["20172018_sva"].merge(final_data["20172018_pp"][list1_merge],left_index=False,right_index=False,suffixes=("","_pp"),how='left',on='Team+day',indicator=False,validate=None,)
merge_20172018=merge_20172018.merge(final_data["20172018_pk"][list2_merge],left_index=False,right_index=False,suffixes=("","_pk"),how='left',on='Team+day',indicator=False,validate=None,)

In [163]:
merge_20182019=final_data["20182019_sva"].merge(final_data["20182019_pp"][list1_merge],left_index=False,right_index=False,suffixes=("","_pp"),how='left',on='Team+day',indicator=False,validate=None,)
merge_20182019=merge_20182019.merge(final_data["20182019_pk"][list2_merge],left_index=False,right_index=False,suffixes=("","_pk"),how='left',on='Team+day',indicator=False,validate=None,)

In [164]:
merge_20192020=final_data["20192020_sva"].merge(final_data["20192020_pp"][list1_merge],left_index=False,right_index=False,suffixes=("","_pp"),how='left',on='Team+day',indicator=False,validate=None,)
merge_20192020=merge_20192020.merge(final_data["20192020_pk"][list2_merge],left_index=False,right_index=False,suffixes=("","_pk"),how='left',on='Team+day',indicator=False,validate=None,)

In [165]:
merge_20202021=final_data["20202021_sva"].merge(final_data["20202021_pp"][list1_merge],left_index=False,right_index=False,suffixes=("","_pp"),how='left',on='Team+day',indicator=False,validate=None,)
merge_20202021=merge_20202021.merge(final_data["20202021_pk"][list2_merge],left_index=False,right_index=False,suffixes=("","_pk"),how='left',on='Team+day',indicator=False,validate=None,)

In [166]:
final_output=pd.concat([merge_20152016,merge_20162017,merge_20172018,merge_20182019,merge_20192020,merge_20202021])

In [168]:
final_output.columns.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False])

### Since Ice Hockey is a low-scoring game. To smoothen out the varience in the data, we are calculating rolling averages of the data. 
###We have calculated 40 an 5 rolling game averages. In the end to include all the opponents a team might encounter, we have decided to use 40 games average. 

In [169]:
final_output["TOI_SVA_TeamMoingAverage_40"]=final_output.groupby('Team')['TOI'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["TOI_SVA_TeamMoingAverage_5"]=final_output.groupby('Team')['TOI'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [170]:
final_output["CA_SVA_TeamMoingAverage_40"]=final_output.groupby('Team')['CA'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["CA_SVA_TeamMoingAverage_5"]=final_output.groupby('Team')['CA'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [171]:
final_output["FF_SVA_TeamMoingAverage_40"]=final_output.groupby('Team')['FF'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["FF_SVA_TeamMoingAverage_5"]=final_output.groupby('Team')['FF'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [172]:
final_output["FA_SVA_TeamMoingAverage_40"]=final_output.groupby('Team')['FA'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["FA_SVA_TeamMoingAverage_5"]=final_output.groupby('Team')['FA'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [173]:
final_output["SF_SVA_TeamMoingAverage_40"]=final_output.groupby('Team')['SF'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["SF_SVA_TeamMoingAverage_5"]=final_output.groupby('Team')['SF'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [174]:
final_output["SA_SVA_TeamMoingAverage_40"]=final_output.groupby('Team')['SA'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["SA_SVA_TeamMoingAverage_5"]=final_output.groupby('Team')['SA'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [175]:
final_output["GA_SVA_TeamMoingAverage_40"]=final_output.groupby('Team')['GA'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["GA_SVA_TeamMoingAverage_5"]=final_output.groupby('Team')['GA'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [176]:
final_output["GF_SVA_TeamMoingAverage_40"]=final_output.groupby('Team')['GF'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["GF_SVA_TeamMoingAverage_5"]=final_output.groupby('Team')['GF'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [177]:
final_output["GA_SVA_TeamMoingAverage_40"]=final_output.groupby('Team')['GA'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["GA_SVA_TeamMoingAverage_5"]=final_output.groupby('Team')['GA'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [178]:
final_output["xGA_SVA_TeamMoingAverage_40"]=final_output.groupby('Team')['xGA'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["xGA_SVA_TeamMoingAverage_5"]=final_output.groupby('Team')['xGA'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [179]:
final_output["xGF_SVA_TeamMoingAverage_40"]=final_output.groupby('Team')['xGF'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["xGF_SVA_TeamMoingAverage_5"]=final_output.groupby('Team')['xGF'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [180]:
final_output["Fenwick%_SVA_TeamMoingAverage_40"]=(final_output["FF_SVA_TeamMoingAverage_40"]/(final_output["FF_SVA_TeamMoingAverage_40"]+final_output["FA_SVA_TeamMoingAverage_40"]))*100
final_output["Fenwick%_SVA_TeamMoingAverage_5"]=(final_output["FF_SVA_TeamMoingAverage_5"]/(final_output["FF_SVA_TeamMoingAverage_5"]+final_output["FA_SVA_TeamMoingAverage_5"]))*100

In [181]:
final_output["GF%_SVA_TeamMoingAverage_40"]=(final_output["GF_SVA_TeamMoingAverage_40"]/(final_output["GF_SVA_TeamMoingAverage_40"]+final_output["GA_SVA_TeamMoingAverage_40"]))*100
final_output["GF%_SVA_TeamMoingAverage_5"]=(final_output["GF_SVA_TeamMoingAverage_5"]/(final_output["GF_SVA_TeamMoingAverage_5"]+final_output["GA_SVA_TeamMoingAverage_5"]))*100

In [182]:
final_output["xGF%_SVA_TeamMoingAverage_40"]=(final_output["xGF_SVA_TeamMoingAverage_40"]/(final_output["xGF_SVA_TeamMoingAverage_40"]+final_output["xGA_SVA_TeamMoingAverage_40"]))*100
final_output["xGF%_SVA_TeamMoingAverage_5"]=(final_output["xGF_SVA_TeamMoingAverage_5"]/(final_output["xGF_SVA_TeamMoingAverage_5"]+final_output["xGA_SVA_TeamMoingAverage_5"]))*100

In [183]:
final_output["SFGF%_SVA_TeamMoingAverage_40"]=(final_output["GF_SVA_TeamMoingAverage_40"]/(final_output["SF_SVA_TeamMoingAverage_40"]))*100
final_output["SFGF%_SVA_TeamMoingAverage_5"]=(final_output["GF_SVA_TeamMoingAverage_5"]/(final_output["SF_SVA_TeamMoingAverage_5"]))*100

In [184]:
final_output["TOI_pp_TeamMoingAverage_40"]=final_output.groupby('Team')['TOI_pp'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["TOI_pp_TeamMoingAverage_5"]=final_output.groupby('Team')['TOI_pp'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [185]:
final_output["GF_pp_TeamMoingAverage_40"]=final_output.groupby('Team')['GF_pp'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["GF_pp_TeamMoingAverage_5"]=final_output.groupby('Team')['GF_pp'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [186]:
final_output["xGF_pp_TeamMoingAverage_40"]=final_output.groupby('Team')['xGF_pp'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["xGF_pp_TeamMoingAverage_5"]=final_output.groupby('Team')['xGF_pp'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [187]:
final_output["TOI_Average_pp_TeamMoingAverage_40"]=final_output.groupby('Team')['TOI_pp'].transform(lambda x: x.rolling(window=40,min_periods=40).mean().shift())
final_output["TOI_Average_pp_TeamMoingAverage_5"]=final_output.groupby('Team')['TOI_pp'].transform(lambda x: x.rolling(window=5,min_periods=5).mean().shift())

In [188]:
final_output["GF_Average_Average_pp_TeamMoingAverage_40"]=final_output["GF_pp_TeamMoingAverage_40"]/final_output["TOI_pp_TeamMoingAverage_40"]
final_output["GF_Average_Average_pp_TeamMoingAverage_5"]=final_output["GF_pp_TeamMoingAverage_5"]/final_output["TOI_pp_TeamMoingAverage_5"]

In [189]:
final_output["xGF_Average_pp_TeamMoingAverage_40"]=final_output["xGF_pp_TeamMoingAverage_40"]/final_output["TOI_pp_TeamMoingAverage_40"]
final_output["xGF_Average_pp_TeamMoingAverage_5"]=final_output["xGF_pp_TeamMoingAverage_5"]/final_output["TOI_pp_TeamMoingAverage_5"]

In [197]:
final_output["TOI_pk_TeamMoingAverage_40"]=final_output.groupby('Team')['TOI_pk'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["TOI_pk_TeamMoingAverage_5"]=final_output.groupby('Team')['TOI_pk'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [191]:
final_output["GA_pk_TeamMoingAverage_40"]=final_output.groupby('Team')['GA_pk'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["GA_pk_TeamMoingAverage_5"]=final_output.groupby('Team')['GA_pk'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [192]:
final_output["xGA_pk_TeamMoingAverage_40"]=final_output.groupby('Team')['xGA_pk'].transform(lambda x: x.rolling(window=40,min_periods=40).sum().shift())
final_output["xGA_pk_TeamMoingAverage_5"]=final_output.groupby('Team')['xGA_pk'].transform(lambda x: x.rolling(window=5,min_periods=5).sum().shift())

In [193]:
final_output["TOI_Average_pk_TeamMoingAverage_40"]=final_output.groupby('Team')['TOI_pk'].transform(lambda x: x.rolling(window=40,min_periods=40).mean().shift())
final_output["TOI_Average_pk_TeamMoingAverage_5"]=final_output.groupby('Team')['TOI_pk'].transform(lambda x: x.rolling(window=5,min_periods=5).mean().shift())

In [194]:
final_output["GA_Average_Average_pk_TeamMoingAverage_40"]=final_output["GA_pk_TeamMoingAverage_40"]/final_output["TOI_pk_TeamMoingAverage_40"]
final_output["GA_Average_Average_pk_TeamMoingAverage_5"]=final_output["GA_pk_TeamMoingAverage_5"]/final_output["TOI_pk_TeamMoingAverage_5"]

In [195]:
final_output["xGA_Average_pk_TeamMoingAverage_40"]=final_output["xGA_pk_TeamMoingAverage_40"]/final_output["TOI_pk_TeamMoingAverage_40"]
final_output["xGA_Average_pk_TeamMoingAverage_5"]=final_output["xGA_pk_TeamMoingAverage_5"]/final_output["TOI_pk_TeamMoingAverage_5"]

In [196]:
final_output['previous_play']=final_output.groupby('Team')['day'].shift()
final_output['previous_play_day']=(final_output['day']).apply(lambda x: pd.to_datetime(x))-(final_output['previous_play']).apply(lambda x: pd.to_datetime(x))

In [198]:
final_output.loc[final_output['previous_play_day']=='1 days','BacktoBack']=1
final_output.loc[final_output['previous_play_day']!='1 days','BacktoBack']=0

In [199]:
final_output['previous_play_day']

0         NaT
1         NaT
2         NaT
3      0 days
4      0 days
        ...  
1899   2 days
1900   3 days
1901   3 days
1902   2 days
1903   2 days
Name: previous_play_day, Length: 16176, dtype: timedelta64[ns]

In [231]:
final_output

,Game,Team,Unnamed: 2,TOI,CF,CA,CF%,FF,FA,FF%,...,xGA_pk_TeamMoingAverage_5,TOI_Average_pk_TeamMoingAverage_40,TOI_Average_pk_TeamMoingAverage_5,GA_Average_Average_pk_TeamMoingAverage_40,GA_Average_Average_pk_TeamMoingAverage_5,xGA_Average_pk_TeamMoingAverage_40,xGA_Average_pk_TeamMoingAverage_5,previous_play,previous_play_day,BacktoBack
0,"2015-09-20 - Panthers 3, Predators 2",FLA,Limited ReportFull Report,62.083333,41,45,47.67,34,37,47.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0.0
1,"2015-09-20 - Panthers 3, Predators 2",NSH,Limited ReportFull Report,62.083333,45,41,52.33,37,34,52.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0.0
2,"2015-09-21 - Oilers 3, Flames 1",CGY,Limited ReportFull Report,60.000000,53,65,44.92,41,47,46.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0.0
3,"2015-09-21 - Oilers 3, Flames 1",CGY,Limited ReportFull Report,60.000000,53,65,44.92,41,47,46.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-21,0 days,0.0
4,"2015-09-21 - Oilers 3, Flames 1",CGY,Limited ReportFull Report,60.000000,53,65,44.92,41,47,46.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-21,0 days,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,"2021-07-02 - Lightning 6, Canadiens 3",T.B,Limited ReportFull Report,60.000000,48,59,44.86,37,46,44.58,...,11.21,60.547917,60.226667,0.040877,0.016604,0.042342,0.037226,2021-06-30,2 days,0.0
1900,"2021-07-05 - Lightning 2, Canadiens 3",MTL,Limited ReportFull Report,63.950000,42,70,37.5,26,51,33.77,...,14.01,61.122083,60.330000,0.047037,0.056357,0.044403,0.046445,2021-07-02,3 days,0.0
1901,"2021-07-05 - Lightning 2, Canadiens 3",T.B,Limited ReportFull Report,63.950000,70,42,62.5,51,26,66.23,...,11.21,60.547917,60.226667,0.040877,0.026566,0.042396,0.037226,2021-07-02,3 days,0.0
1902,"2021-07-07 - Canadiens 0, Lightning 1",MTL,Limited ReportFull Report,60.000000,42,47,47.19,33,38,46.48,...,15.43,61.220833,61.120000,0.047369,0.058901,0.045507,0.050491,2021-07-05,2 days,0.0


### Using NHL API to import season start and end dates.

In [201]:
season_dates=json.loads(requests.get("https://statsapi.web.nhl.com/api/v1/seasons").content)

In [202]:
season_dates_output=[]
for i in range(len(years)):
  if(len([i1 for i1 in season_dates["seasons"] if i1["regularSeasonEndDate"][0:4]==years[i][4:]])>0):
    season_dates_output.append([i1 for i1 in season_dates["seasons"] if i1["regularSeasonEndDate"][0:4]==years[i][4:]])

### Using hockey scrappr to acquire game results for each season

In [203]:
season_results={}
for i in range(len(season_dates_output)):
  time.sleep(3)
  dummy_list21=[]
  season_iteration=hockey_scraper.scrape_schedule(season_dates_output[i][0]["regularSeasonStartDate"],season_dates_output[i][0]["seasonEndDate"])
  for j in range(len(season_iteration)):
    if(season_iteration["home_score"][j]>season_iteration["away_score"][j]):
      dummy_list21.append(1)
    else:
      dummy_list21.append(0)
  season_iteration["Host_won"]=dummy_list21
  season_iteration["Team+day_Host"]=season_iteration["home_team"]+' + '+season_iteration["date"]
  season_iteration["Team+day_guest"]=season_iteration["away_team"]+' + '+season_iteration["date"]
  season_results[season_dates_output[i][0]["regularSeasonStartDate"][0:4]+"_"+season_dates_output[i][0]["seasonEndDate"][0:4]]=season_iteration

Scraping the schedule between 2015-10-07 and 2016-06-12
Scraping the schedule between 2016-10-12 and 2017-06-11
Scraping the schedule between 2017-10-04 and 2018-06-07
Scraping the schedule between 2018-10-03 and 2019-06-15
Scraping the schedule between 2019-10-02 and 2020-09-29
Scraping the schedule between 2021-01-13 and 2021-07-15


In [204]:
def final_combine(input1,input2,input3,input4,indexes):
  home='home_'
  guest='guest_'
  outpt=input1.merge(input2[indexes].add_prefix(home),left_on='Team+day_Host', right_on = 'home_Team+day', how = 'left').drop(columns ='home_Team+day')
  outpt=outpt.merge(input2[indexes].add_prefix(guest),left_on='Team+day_guest',right_on='guest_Team+day',how ='left').drop(columns='guest_Team+day')
  outpt=outpt.merge(input3[input4].add_prefix(home),left_on='Team+day_Host',right_on='home_Team+day',how = 'left')
  outpt=outpt.merge(input3[input4].add_prefix(guest),left_on='Team+day_guest',right_on='guest_Team+day',how = 'left')
  outpt['match_BacktoBack']=np.select([((outpt['home_BacktoBack'] == 0) & (outpt['guest_BacktoBack'] == 0)),((outpt['home_BacktoBack']==1)&(outpt['guest_BacktoBack']==0)),((outpt['home_BacktoBack']==0)&(outpt['guest_BacktoBack']==1)),((outpt['home_BacktoBack']==1)&(outpt['guest_BacktoBack']==1))],['Neither','Home_only','guest_only','Both'])
  outpt['year']=np.select([((outpt['date'] >= season_dates_output[0][0]["regularSeasonStartDate"]) & (outpt['date'] <= season_dates_output[0][0]["seasonEndDate"])),
                  ((outpt['date'] >= season_dates_output[1][0]["regularSeasonStartDate"]) & (outpt['date'] <= season_dates_output[1][0]["seasonEndDate"])),
                  ((outpt['date'] >= season_dates_output[2][0]["regularSeasonStartDate"]) & (outpt['date'] <= season_dates_output[2][0]["seasonEndDate"])),
                  ((outpt['date'] >= season_dates_output[3][0]["regularSeasonStartDate"]) & (outpt['date'] <= season_dates_output[3][0]["seasonEndDate"])),
                  ((outpt['date'] >= season_dates_output[4][0]["regularSeasonStartDate"]) & (outpt['date'] <= season_dates_output[4][0]["seasonEndDate"])),
                  ((outpt['date'] >= '2020-04-01') & (outpt['date'] <= season_dates_output[5][0]["seasonEndDate"]))],['20152016','20162017','20172018','20182019','20192020','20202021'])
  return outpt
  

In [205]:
x1=[]
years2=["2015_2016","2016_2017","2017_2018","2018_2019","2019_2020","2021_2021"]

In [206]:
for i in range(len(years2)):
  x1.append(final_combine(season_results[years2[i]],goalkeeper_fulldata,final_output,final_output.columns,['Team+day','fullName','fwkSV%_GoalKeeper','GSAx/60_GoalKeeper','HDCSV%__GoalKeeper']))

In [207]:
final_scrapped_data=pd.concat([x1[0],x1[1],x1[2],x1[3],x1[4],x1[5]])

In [208]:
y=['guest_fwkSV%_GoalKeeper','guest_GSAx/60_GoalKeeper','guest_HDCSV%__GoalKeeper','home_fwkSV%_GoalKeeper','home_GSAx/60_GoalKeeper','home_HDCSV%__GoalKeeper']
y1=[ff_calc,GSAx60_calc,HDCSV_calc,ff_calc,GSAx60_calc,HDCSV_calc]

In [209]:
for i in range(len(y)):
  final_scrapped_data[y[i]]=np.where(final_scrapped_data[y[i]].isna(),y1[i],final_scrapped_data[y[i]])

### Encoding team name with team label for each team

In [210]:
labelencoder=LabelEncoder()

In [211]:
final_scrapped_data['home_team_label']=labelencoder.fit_transform(final_scrapped_data['home_team'])

In [212]:
final_scrapped_data['away_team_label']=labelencoder.fit_transform(final_scrapped_data['away_team'])

In [213]:
final_scrapped_data.head()

,game_id,date,venue,home_team,away_team,start_time,home_score,away_score,status,Host_won,...,guest_GA_Average_Average_pk_TeamMoingAverage_5,guest_xGA_Average_pk_TeamMoingAverage_40,guest_xGA_Average_pk_TeamMoingAverage_5,guest_previous_play,guest_previous_play_day,guest_BacktoBack,match_BacktoBack,year,home_team_label,away_team_label
0,2015020001,2015-10-07,Air Canada Centre,TOR,MTL,2015-10-07 23:00:00,1,3,Final,0,...,0.059016,NaN,0.034656,2015-10-03,4 days,0.0,Neither,20152016,28,16
1,2015020002,2015-10-07,United Center,CHI,NYR,2015-10-08 00:00:00,2,3,Final,0,...,0.048977,NaN,0.047899,2015-09-30,7 days,0.0,Neither,20152016,8,20
2,2015020003,2015-10-07,Scotiabank Saddledome,CGY,VAN,2015-10-08 02:00:00,1,5,Final,0,...,0.033102,NaN,0.044389,2015-10-03,4 days,0.0,Neither,20152016,7,28
3,2015020004,2015-10-07,STAPLES Center,L.A,S.J,2015-10-08 02:30:00,1,5,Final,0,...,0.029515,NaN,0.041484,2015-10-03,4 days,0.0,Neither,20152016,15,24
4,2015020005,2015-10-08,TD Garden,BOS,WPG,2015-10-08 23:00:00,2,6,Final,0,...,0.059068,NaN,0.043809,2015-10-03,5 days,0.0,Neither,20152016,3,30


In [217]:
final_scrapped_data.to_csv('final_scrapped_data.csv')

In [215]:
final_scrapped_data.columns

Index(['game_id', 'date', 'venue', 'home_team', 'away_team', 'start_time',
       'home_score', 'away_score', 'status', 'Host_won',
       ...
       'guest_GA_Average_Average_pk_TeamMoingAverage_5',
       'guest_xGA_Average_pk_TeamMoingAverage_40',
       'guest_xGA_Average_pk_TeamMoingAverage_5', 'guest_previous_play',
       'guest_previous_play_day', 'guest_BacktoBack', 'match_BacktoBack',
       'year', 'home_team_label', 'away_team_label'],
      dtype='object', length=268)